# BERTweet.ipynb

### Designation: Model Generation Script

    Purpose: to train a single BERTweet model on our dataset for training ('Tweets.csv') at 90:10 train-test split

- Requirements:
    
    Packages: tensorflow, pandas, matplotlib, transformers, sklearn, os

    Datasets (csv's): Tweets.csv

- This program will require an internet connection, as it will download the model and tokenizer from the HuggingFace model repository.

- Saved model-weight name (output): bertweet9010.h5
    - Please note, all files referenced (input and output) will all be on the folder-level.

### A note on BERTweet, Tensorflow and GPU:

- This is a RoBERTa model, which means it makes predictions of each word based on ALL words around it in the sentence. This has the following consequences, aside from achieving high accuracy

    - It has a LOT of weights to be trained. hence a GPU is strongly recommended, if I cannot 'require' it.

    - It is very prone to overfitting, (although slightly better than BERT), and as such, the way to tune this model is to run 1 epoch on as many unique queries as possible.

    - It will take up a lot of storage. We would even observe training speed differences if it was on a hard drive vs. if it was on an SSD.

- For Tensorflow:
    
    - This model is compiled with Tensorflow 2.8+, with CUDA toolkit version 11.6 with its corresponding cuDNN and other nVidia required databases on python 3.8.10

    - Training time reference: 12 minutes (GPU) - 1 epoch, batch size 32, 90:10 train-test ratio, RTX 3060, Ryzen 5 5600x, 8GB ram assigned, WSL-2 ubuntu 20.04 LTS.



## 1. TensorFlow Standalone Setup

- To get tensorflow working with the correct device, we import it first before we load the model.

- 'useCPU' argument will disable CUDA and force TensorFlow to run with CPU only.
    
    - there are many 'with tf.device('/GPU:0'):' casts, forcing TensorFlow to run with CPU only here will not raise an error. 
        
        TensorFlow will use CPU even when the code tells it to use GPU, as there is no GPU detected.

            Tip: Restart the kernel before running the program (again), so no variable collision happens, and it will ensure that there are RAM/VRAM available to use.

In [1]:
useCPU = False #Choose whether to use CPU or GPU for running the program

import tensorflow as tf
import os
if useCPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
tf.config.list_physical_devices('GPU')

Num GPUs Available:  1


2022-06-02 18:49:25.485842: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-02 18:49:25.742786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-02 18:49:25.743087: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.


## 2. Importing, downloading, and Building the model

- The model and tokenizer are quite large, so make sure enough drive space is provided in C drive, and a fast, reliable internet connection is available.

- We have specified that we want, from HuggingFace, a BERTweet tokenizer (using AutoModel Casting), and the BerTweet (RoBERTa) model with a sequence classification head attached to it.

    - And we specified that we are doing a 3-label classification.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import InputExample, InputFeatures
with tf.device('/GPU:0'):
    model = TFAutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base",num_labels=3,problem_type="multi_label_classification")
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base",num_labels=3)
    model.summary()

/home/acthegreat/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-06-02 18:49:29.025403: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-02 18:49:29.026916: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-02 18:49:29.027208: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your 

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 134309376 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592899    
 ficationHead)                                                   
                                                                 
Total params: 134,902,275
Trainable params: 134,902,275
Non-trainable params: 0
_________________________________________________________________


## 3. Read in dataset: Tweets.csv (our dataset for training purposes), and clean up the dataset

- textID and selected_text will not matter for this program.

In [4]:
dataset = pd.read_csv('../Dataset/Tweets.csv', encoding='ISO-8859-1')
dataset_drop = dataset.drop(['textID', 'selected_text'], axis=1)
dataset_drop

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
...,...,...
27476,wish we could come see u on Denver husband l...,negative
27477,I`ve wondered about rake to. The client has ...,negative
27478,Yay good for both of you. Enjoy the break - y...,positive
27479,But it was worth it ****.,positive


### 3.1. Extract and encode the dataset's label column into number-category encoding.

- 0 is negative, 1 is neutral, 2 is positive.

In [5]:
datasetSentimentEncode = dataset_drop['sentiment'].apply(lambda c: 0 if c == 'negative' else (1 if c=='neutral' else 2))
datasetSentimentEncode

0        1
1        0
2        0
3        0
4        0
        ..
27476    0
27477    0
27478    2
27479    2
27480    1
Name: sentiment, Length: 27481, dtype: int64

## 4. Compiling Training/Test split dataframes

- 90:10 seeded split, and dataframes are named in a specific way so we can use the loader functions

In [6]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(dataset_drop['text'].astype(str), datasetSentimentEncode, test_size=0.1, random_state=21)
trainDF = pd.DataFrame()
testDF = pd.DataFrame()
trainDF['DATA_COLUMN'] = xtrain
trainDF['LABEL_COLUMN'] = ytrain
testDF['DATA_COLUMN'] = xtest
testDF['LABEL_COLUMN'] = ytest
trainDF,testDF

(                                             DATA_COLUMN  LABEL_COLUMN
 8775                                   blastinggg music.             1
 8885    If it`s any consolation, you`re definitely on...             2
 22325                    fun day with boo. short but fun             2
 13024   Blow me away it IS raining harder here. Yay y...             1
 17426   Lame remarks like 'I wonder if they like blon...             1
 ...                                                  ...           ...
 16432                                   FC is back dear.             1
 8964    tea...  Mmmm crispy but no cake  Have headpho...             1
 5944                       thankyou very much, you rock!             2
 5327                                i looking at failure             1
 15305   happy mommas day . ging is so lucky to have a...             2
 
 [24732 rows x 2 columns],
                                              DATA_COLUMN  LABEL_COLUMN
 26493  I started X-Slimmer at eigh

### 4.1. (Optional) Saving the seeded testing/training data splits.

- this is done for various verification purposes.

In [7]:
#testDF.to_csv('testingData.csv')
#trainDF.to_csv('trainingData.csv')

## 5. Converting dataframes into supported input format for the AI

- defining methods of conversion below

- The conversion will first convert the dataframe into a list of InputExample Objects

    - InputExample: Hugging Face-provided object-enclosure for data.

- Then it will convert the list of InputExample objects into a TensorFlow Tensor Dataset, with a list of text IDs and a list of labels.

    - The tokenizer is used to convert the plain text into a list of vectors (numbers, textIDs), that the machine understands.



In [19]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


### 5.1. The call of the functions, and batching

- We will transform both train and test data separately.

In [20]:
DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'
train_InputExamples, validation_InputExamples = convert_data_to_examples(trainDF, testDF, DATA_COLUMN, LABEL_COLUMN)
with tf.device('/GPU:0'):
    train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
    train_data = train_data.shuffle(100).batch(32)

    validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
    validation_data = validation_data.batch(32)

/home/acthegreat/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


## 6. Train the model

- Compiling the model, which we are using Adam, and we specify the learning rate here.

- We run model.fit() (the train command), and we specify the epoch we run here.

In [22]:
model.layers[0].trainable = True #Testing: whether training the head only or the entire model would result in a better model.
with tf.device('/GPU:0'):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), #default: 3e-5
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
    out = model.fit(train_data, epochs=1, validation_data=validation_data)

773/773 [==============================] - 439s 554ms/step - loss: 0.5660 - accuracy: 0.7644 - val_loss: 0.4612 - val_accuracy: 0.8068


## 7. (Optional) Save Weights

- If the training result is satisfactory, uncomment the code below and run the cell.
    - it will grab the model weight that is still stored in the kernel and save it.

In [25]:
#model.save_weights('bertweet9010.h5')